# Feedforward neuraal netwerk met _back propagation_

## Leerdoelen
- Feedforward neural network implementeren met PyTorch
- PyTorch nn.Module gebruiken voor model definitie
- ReLU activatie en Softmax output voor multi-class classificatie
- Cross-entropy loss gebruiken met PyTorch
- Model trainen met PyTorch optimizer (SGD)
- Model performance evalueren op 3-class classificatie
- PyTorch's ingebouwde tools gebruiken voor visualisatie en debugging

In [ ]:
from sklearn.datasets import load_iris

## 1. Data Loading en Exploratie

## ✍️

We gebruiken de volledige Iris dataset met **alle 3 klassen**:
- **Setosa** (klasse 0)
- **Versicolor** (klasse 1)
- **Virginica** (klasse 2)

In [ ]:
# Load the complete Iris dataset
iris = load_iris()

## 2. Data Voorbereiding

## ✍️

We bereiden de data voor door:
1. **Train/test split** (80/20)
2. **Feature standardization** (mean=0, std=1)
3. **One-hot encoding** van de target labels voor multi-class classificatie

## 3. Neural Network Architectuur met PyTorch

## ✍️

We bouwen een feedforward neural network met PyTorch's `nn.Module`:

**Input Layer → Hidden Layer → Output Layer**

- **Input**: 4 features (sepal length, sepal width, petal length, petal width)
- **Hidden Layer**: 8 neurons met ReLU activatie
- **Output Layer**: 3 neurons (voor 3 klassen)

### Loss Functie

**CrossEntropyLoss** in PyTorch (incl. _Softmax_):
$$
\mathcal{L}_{CE} = -\frac{1}{n} \sum_{i=1}^{n} \log\left(\frac{e^{z_{y_i}}}{\sum_{j=1}^{K} e^{z_j}}\right)
$$

## 4. Loss Functie en Optimizer

## ✍️

PyTorch maakt training veel eenvoudiger:
- **Loss functie**: `nn.CrossEntropyLoss()` combineert Softmax + Cross-Entropy
- **Optimizer**: `optim.SGD()` implementeert gradient descent met momentum optie
- **Backpropagation**: Gebeurt automatisch met `loss.backward()`

## 5. Training het Neural Network met PyTorch

## ✍️

De training loop is veel eenvoudiger met PyTorch:
1. **Forward pass**: `outputs = model(inputs)`
2. **Compute loss**: `loss = criterion(outputs, targets)`
3. **Backward pass**: `loss.backward()` (automatische backpropagation!)
4. **Update weights**: `optimizer.step()`

## 6. Visualisatie van Training Proces

## ✍️

We visualiseren hoe de loss en accuracy evolueren tijdens training.

## 7. Model Evaluatie

## 🎯

Laten we de voorspellingen analyseren en een confusion matrix maken om te zien hoe goed het model elke klasse herkent.

## 8. Decision Boundaries Visualisatie

## 🎯

We visualiseren de decision boundaries van het neural network in 2D (gebruikmakend van de twee belangrijkste features).

## 9. Model Inspectie met PyTorch

## 🎯

PyTorch biedt handige tools om het model te inspecteren.